In [1]:
input_file = "input_files/day_14.txt"

with open(input_file) as lines:
    data = lines.read().splitlines()


## Part One
Use Numpy to make indexing a little nicer and faster - it can sort slices in place.

1. Build a list of ranges between the square
2. Iterate over the ranges and sort to move the round rocks
3. Calculate score

In [2]:
import numpy as np
import re

rx = re.compile(r'([O|\.]+)')

row_ranges = [(r, m.span()) for r, line in enumerate(data) for m in rx.finditer(line)]
col_ranges = [(r, m.span()) for r, line in enumerate(map(''.join, zip(*data))) for m in rx.finditer(line)]

v = {'.': 0, 'O': 1, '#': 0 }
arr = np.array([[v[c] for c in line] for line in data])


def south(arr):
    for col, (start, stop) in col_ranges:
        arr[start:stop, col].sort()
def north(arr):
    for col, (start, stop) in col_ranges:
        arr[stop - 1:start - 1 if start else None : -1, col].sort()
def east(arr):
    for row, (start, stop) in row_ranges:
        arr[row, start:stop].sort()
def west(arr):
    for row, (start, stop) in row_ranges:
        arr[row, stop - 1:start - 1 if start else None : -1].sort()

def score(arr):
    return (arr * np.arange(arr.shape[0], 0, -1).reshape(-1, 1)).sum()

north(arr)
print("Part one:", score(arr))


Part one: 109638


# Part Two

1. Make the assumption that this will be a repeating cycle of some sort because of the relatively small state space.
2. Keep track of a hash (numpy arrays are not hashable, but their backing bytes are) of each state and run until we see that state again.
3. Extrapolate which small iteration matches the billionth. 

In [3]:
arr = np.array([[v[c] for c in line] for line in data])

def cycle(arr):
    north(arr)
    west(arr)
    south(arr)
    east(arr)

def find_cycle(max_search = 200):
    seen = {arr.data.tobytes(): 0}

    for i in range(1, 300):
        cycle(arr)
        b = arr.data.tobytes()
        if b in seen:
            prefix = seen[b]
            modulus = i - prefix
            print("prefix:", seen[b], "cycle:", i - seen[b])
            break
        seen[arr.data.tobytes()] = i
    return prefix, modulus

prefix, modulus = find_cycle()

iteration = (1_000_000_000 - prefix) % modulus + prefix

arr = np.array([[v[c] for c in line] for line in data])

for _ in range(iteration):
    cycle(arr)
    
print("Part Two:", score(arr))

prefix: 122 cycle: 21
Part Two: 102657
